## **Промежуточная аттестация 1.1**

### Считываем данные

In [1]:
import pandas as pd
import numpy as np

Данные об аккумуляторных газонокосилках:
* Название газонокосилки — `NAME`
* Ширина скашиваемой полосы — `WIDTH`
* Стоимость — `PRICE`
* Мощность — `POWER`
* Объем травосборника — `GRASS`
* Площадь скашиваемой поверхности на одном заряде аккумулятора — `AREA`
* Описание газонокосилки — `DESCRIPTION`

In [2]:
df = pd.read_csv('https://dc-edu.itmo.ru/assets/courseware/v1/4f5098b62b9c0e98d07d0f85da65a185/asset-v1:ITMO+bonus_track_methods+2023+type@asset+block/lawnmower_var_343286.csv')
df

,NAME,WIDTH,PRICE,POWER,GRASS,AREA,DESCRIPTION
0,Bear YEX635,39.0,45000.0,1.25,60.0,350.0,Средняя ширина скашиваемой полосы на данной га...
1,Wolf ISB956,42.0,27000.0,1.00,35.0,350.0,Широкая полоса скашивания на данной газонокоси...
2,Boch LAQ128,37.0,13000.0,1.35,55.0,450.0,Средняя ширина скашиваемой полосы на данной га...
3,Bear XAK811,NaN,24000.0,1.70,65.0,350.0,Газонокосилка с узкой шириной скашиваемой поло...
4,Worx KCF632,42.0,10000.0,1.60,40.0,450.0,Широкая полоса скашивания на данной газонокоси...
...,...,...,...,...,...,...,...
195,Cooper ZNA551,42.0,24000.0,1.85,65.0,450.0,Широкая полоса скашивания на данной газонокоси...
196,Nakita OSD693,43.0,32000.0,0.85,30.0,NaN,Широкая полоса скашивания на данной газонокоси...
197,Worx VGN432,42.0,12000.0,1.65,30.0,400.0,Широкая полоса скашивания на данной газонокоси...
198,Wolf VJE655,44.0,32000.0,1.30,75.0,500.0,Широкая полоса скашивания на данной газонокоси...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NAME         200 non-null    object 
 1   WIDTH        180 non-null    float64
 2   PRICE        199 non-null    float64
 3   POWER        198 non-null    float64
 4   GRASS        195 non-null    float64
 5   AREA         196 non-null    float64
 6   DESCRIPTION  200 non-null    object 
dtypes: float64(5), object(2)
memory usage: 11.1+ KB


## Заполнение пропущенных значений в `WIDTH`

In [4]:
# ищем медиану для диапазона ширины от 30 до 35
median_30_35 = df[(df['WIDTH'] >= 30) & (df['WIDTH'] <= 35)]['WIDTH'].median()
median_30_35

33.0

In [5]:
# ищем округленное до целого среднее арифметическое для диапазона ширины от 36 до 40
mean_36_40 = round(df[(df['WIDTH'] >= 36) & (df['WIDTH'] <= 40)]['WIDTH'].mean())
mean_36_40

38

In [6]:
# ищем медиану для диапазона ширины от 41 до 45
median_41_45 = df[(df['WIDTH'] >= 41) & (df['WIDTH'] <= 45)]['WIDTH'].median()
median_41_45

43.0

In [7]:
# восстанавливаем пропущенные значения
def restore_width(row):
  if pd.isna(row['WIDTH']):
    description = row['DESCRIPTION'].lower()
    if any(word in description for word in ['узкая', 'узкие', 'узкую', 'узкой']):
      return median_30_35
    elif 'средняя ширина' in description or 'ширина средняя' in description:
      return mean_36_40
    elif all(word in description for word in ['широкая', 'полоса']) or all(word in description for word in ['широкие', 'полосы']):
      return median_41_45
  return row['WIDTH']

df['WIDTH'] = df.apply(restore_width, axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NAME         200 non-null    object 
 1   WIDTH        200 non-null    float64
 2   PRICE        199 non-null    float64
 3   POWER        198 non-null    float64
 4   GRASS        195 non-null    float64
 5   AREA         196 non-null    float64
 6   DESCRIPTION  200 non-null    object 
dtypes: float64(5), object(2)
memory usage: 11.1+ KB


In [9]:
round(df['WIDTH'].mean(), 2)

37.54

## Избавляемся от *экстремальных* выбросов в `PRICE`

In [10]:
Q1 = df['PRICE'].quantile(0.25) # значения первого квартиля
Q3 = df['PRICE'].quantile(0.75) # значения третьего квартиля
# размах квартилей
IQR = Q3 - Q1

# границы для фильтрации выбросов
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

df = df[(df['PRICE'] >= lower_bound) & (df['PRICE'] <= upper_bound)]

## Удаляем оставшиеся пропущенные значения

In [11]:
df = df.dropna()

In [12]:
len(df)

181

In [13]:
round(df['PRICE'].mean(), 2)

26988.95

## Проводим *экспоненциальную* нормировку числовых параметров

In [14]:
# выберем только числовые признаки таблицы
numeric_columns = df.select_dtypes(include=['number']).columns
# проведем нормировку
df[numeric_columns] = 1 - np.exp(1 - df[numeric_columns] / df[numeric_columns].min())

In [15]:
round(df[df['NAME'] == 'Wolf NAF225'][numeric_columns], 2)

,WIDTH,PRICE,POWER,GRASS,AREA
146,0.26,0.85,0.28,0.18,0.63


## Строим целевую функцию

* Нормированная ширина скашиваемой полосы с коэффициентом 3
* Стоимость как слагаемое вида (1 - нормированная стоимость) с коэффициентом 2
* Нормированную мощность с коэффициентом 5
* Нормированный объём травосборника с коэффициентом 8
* Нормированную площадь скашиваемой поверхности с коэффициентом 3

In [16]:
df['FUNC'] = 3 * df['WIDTH'] + (1 - df['PRICE']) * 2 + 5 * df['POWER'] + 8 * df['GRASS'] + 3 * df['AREA']

In [17]:
df = df.sort_values(by=['FUNC'], ascending=False)
df.head(3)

,NAME,WIDTH,PRICE,POWER,GRASS,AREA,DESCRIPTION,FUNC
113,Bear HDS774,0.124827,0.117503,0.753403,0.864665,0.826226,Газонокосилка с узкой шириной скашиваемой поло...,15.302485
46,Cooper WJN287,0.259182,0.527633,0.698806,0.864665,0.776870,Средняя ширина скашиваемой полосы на данной га...,14.464235
180,Boch ZPL569,0.351656,0.747160,0.769307,0.798103,0.776870,Широкая полоса скашивания на данной газонокоси...,14.122618
